For exporting to PDF:
```bash
apt-get install pandoc 
```

Install dependencies:
```bash
pip install bokeh
pip install jupyter_bokeh
```


In [109]:

import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import re
from numpy import NaN

ten_thousand_2_min = "./full-stack-test/iam_load_test_1500_15min.csv"
df = pd.read_csv(ten_thousand_2_min, low_memory=False)
if "tracked=true" in df.extra_tags.unique():
    df = df[df["extra_tags"]=="tracked=true"]
df["timestamp"] = pd.to_datetime(df.timestamp, unit='s')

output_notebook()

df

Loading BokehJS ...

,metric_name,timestamp,metric_value,check,error,error_code,expected_response,group,method,name,proto,scenario,service,status,subproto,tls_version,url,extra_tags,metadata
0,http_reqs,2024-05-10 12:50:43,1.000000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
1,http_req_duration,2024-05-10 12:50:43,8.215000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
2,http_req_blocked,2024-05-10 12:50:43,2.027000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
3,http_req_connecting,2024-05-10 12:50:43,0.248000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
4,http_req_tls_handshaking,2024-05-10 12:50:43,0.000000,NaN,NaN,NaN,True,::setup,GET,http://localhost:8123/generate,HTTP/1.1,NaN,NaN,200.0,NaN,NaN,http://localhost:8123/generate,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951326,data_sent,2024-05-10 13:06:14,2231.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN
951327,data_received,2024-05-10 13:06:14,2178.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN
951328,data_sent,2024-05-10 13:06:14,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
951329,data_received,2024-05-10 13:06:14,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:

def extract_url_prefix(url):
    if url is NaN or 'localhost' in url:
        return None
    pattern = r"(.*)(0x[a-fA-F0-9]{40})"
    match = re.match(pattern, url)
    return match.group(1) if match else url

df['normalized_url'] = df['url'].apply(extract_url_prefix)
df['normalized_url']

df = df[df['normalized_url'].notna()]

In [111]:
df.metric_name.unique()

array(['http_reqs', 'http_req_duration', 'http_req_blocked',
       'http_req_connecting', 'http_req_tls_handshaking',
       'http_req_sending', 'http_req_waiting', 'http_req_receiving',
       'http_req_failed'], dtype=object)

In [112]:
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [113]:
df[df.metric_name == "http_reqs"].groupby(by=["status", "normalized_url", "method"]).count()

metric_name  \
status normalized_url                                     method                
0.0    https://iam.staging.passport.gitcoin.co/api/v0.... POST           1398   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            531   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            570   
       https://iam.staging.passport.gitcoin.co/static/... GET            1103   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST          23972   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           2397   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          22430   
       https://iam.staging.passport.gitcoin.co/static/... GET            6403   
504.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST           3985   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           1248   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            948   
       https://iam.staging.passport.gitcoin.co/static/... GET            3207   

                                                                  timestamp  \
status normalized_url                                     method              
0.0    https://iam.staging.passport.gitcoin.co/api/v0.... POST         1398   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          531   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          570   
       https://iam.staging.passport.gitcoin.co/static/... GET          1103   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST        23972   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST         2397   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        22430   
       https://iam.staging.passport.gitcoin.co/static/... GET          6403   
504.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST         3985   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST         1248   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST          948   
       https://iam.staging.passport.gitcoin.co/static/... GET          3207   

                                                                  metric_value  \
status normalized_url                                     method                 
0.0    https://iam.staging.passport.gitcoin.co/api/v0.... POST            1398   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST             531   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST             570   
       https://iam.staging.passport.gitcoin.co/static/... GET             1103   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST           23972   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            2397   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST           22430   
       https://iam.staging.passport.gitcoin.co/static/... GET             6403   
504.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST            3985   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST            1248   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST             948   
       https://iam.staging.passport.gitcoin.co/static/... GET             3207   

                                                                  check  \
status normalized_url                                     method          
0.0    https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/static/... GET         0   
200.0  https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   
       https://iam.staging.passport.gitcoin.co/api/v0.... POST        0   


In [114]:
# df['normalized_url'] = df.url.str.extract(r'(?<=http://localhost:8080)(.*)')
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [115]:
df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    # "slate gray",
    # "forest green",
    "lavender",
    "coral",
    "turquoise",
    "gold"
]


color = iter(colors)
p = figure(title="Duration (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        # p.line(group.index, group["metric_value"]["min"], legend_label=f"min - {url}", line_width=2, color=next(color))
        # p.line(group.index, group["metric_value"]["max"], legend_label=f"max - {url}", line_width=2, color=next(color))
        p.line(group.index, group["metric_value"]["mean"], legend_label=f"mean - {url}", line_width=2, color=next(color))


show(p)


In [116]:

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]


p = figure(title="Req / second (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        print(idx)
        p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])


show(p)

1
4
7
10


In [117]:

from math import pi
from bokeh.models import DatetimeTickFormatter
from collections import defaultdict

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

# dfts = dft.groupby(by=["url"]).resample("1Min").agg({
#     "metric_value": ["min", "max", "mean", "count"]
# })

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]

p = figure(title="Req / second (status != 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

p.xaxis.major_label_orientation = pi/4

# Initialize dictionaries for tracking sums per URL
url_200_sums = defaultdict(int)
url_non_200_sums = defaultdict(int)

# Loop through the grouped data and collect sums for each URL
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    # Sum up counts for each status
    total_count = group["metric_value"]["count"].sum()
    
    # Check if the status is 200 or not and add to the appropriate sum
    if int(status) == 200:
        url_200_sums[url] += total_count
    else:
        url_non_200_sums[url] += total_count

# Print summary for each individual URL
for url in url_200_sums.keys() | url_non_200_sums.keys():
    sum_200 = url_200_sums[url]
    sum_non_200 = url_non_200_sums[url]
    total = sum_200 + sum_non_200
    success_rate = (float(sum_200) / total if total > 0 else 0) * 100

    print(f"URL: {url}")
    print(f"   200       {sum_200}")
    print(f"   non-200   {sum_non_200}")
    print(f"   success {success_rate:.2f}%\n")

# Optional: If you also want to show the plots
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])

show(p)

URL: https://iam.staging.passport.gitcoin.co/api/v0.0.0/challenge
   200       23972
   non-200   5383
   success 81.66%

URL: https://iam.staging.passport.gitcoin.co/api/v0.0.0/verify
   200       22430
   non-200   1518
   success 93.66%

URL: https://iam.staging.passport.gitcoin.co/static//providerBitMapInfo.json
   200       6403
   non-200   4310
   success 59.77%

URL: https://iam.staging.passport.gitcoin.co/api/v0.0.0/check
   200       2397
   non-200   1779
   success 57.40%

